https://opendata.vancouver.ca/explore/?disjunctive.features&disjunctive.theme&disjunctive.keyword&disjunctive.data-owner&disjunctive.data-team&sort=modified

https://opendata.vancouver.ca/api/v2/console

In [1]:
#Import packages

import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import seaborn as sns
import matplotlib.pyplot as plt

## Working With Building Permits Data

In [2]:
url = 'https://opendata.vancouver.ca/api/v2/catalog/datasets/issued-building-permits/exports/geojson?limit=-1&offset=0&timezone=UTC'
data = requests.get(url)
permits_df = gpd.GeoDataFrame().from_features(data.json())

In [3]:
permits_df.head()

,geometry,permitnumber,permitnumbercreateddate,issuedate,permitelapseddays,projectvalue,typeofwork,address,projectdescription,permitcategory,...,applicantaddress,propertyuse,specificusecategory,buildingcontractor,buildingcontractoraddress,issueyear,geolocalarea,yearmonth,geo_point_2d,geom
0,POINT (-123.09119 49.22429),BP-2018-01711,2018-03-29,2018-07-16,109,10000.0,Addition / Alteration,"6625 FRASER STREET, Vancouver, BC V5X 3T6",Field Review - Addition / Alteration - Interio...,Renovation - Commercial/ Mixed Use - Lower Com...,...,"2265 East 49th Avenue\r\nVancouver, BC V5P1T9",[Office Uses],[General Office],None,None,2018,Sunset,2018-07,"{'lon': -123.0911901, 'lat': 49.2242876}",NaN
1,POINT (-123.12880 49.28819),BP-2020-03206,2020-10-15,2021-04-16,183,50000.0,Addition / Alteration,"738 BROUGHTON STREET, Vancouver, BC V6G 3A7",High Density Housing / Commercial - Addition /...,None,...,"1330 Granville\r\nVancouver, BC V6Z1M7",[Dwelling Uses],[Multiple Dwelling],None,None,2021,West End,2021-04,"{'lon': -123.1288017, 'lat': 49.2881856}",NaN
2,POINT (-123.13701 49.29288),BP-2019-00328,2019-01-24,2019-01-31,7,60000.0,Addition / Alteration,"1905 ROBSON STREET, Vancouver, BC V6G 1E6",High Density Housing / Commercial - Addition /...,None,...,"1330 Granville\r\nVancouver, BC V6Z1M7",[Dwelling Uses],[Multiple Dwelling],None,None,2019,West End,2019-01,"{'lon': -123.1370062, 'lat': 49.2928799}",NaN
3,POINT (-123.13715 49.28095),BP-2020-03502,2020-11-06,2021-01-27,82,50000.0,Addition / Alteration,"1260 HARWOOD STREET, Vancouver, BC V6E 1S4",High Density Housing / Commercial - Addition /...,None,...,"1330 Granville\r\nVancouver, BC V6Z1M7",[Dwelling Uses],[Multiple Dwelling],None,None,2021,West End,2021-01,"{'lon': -123.1371518, 'lat': 49.2809495}",NaN
4,POINT (-123.13825 49.28318),BP-2019-02718,2019-06-18,2019-11-14,149,35000.0,Addition / Alteration,"1434 BURNABY STREET, Vancouver, BC V6G 1W8",High Density Housing / Commercial - Addition /...,None,...,"1330 Granville\r\nVancouver, BC V6Z1M7",[Dwelling Uses],[Multiple Dwelling],None,None,2019,West End,2019-11,"{'lon': -123.1382513, 'lat': 49.2831764}",NaN


### Data Cleaning & Preparation

In [4]:
#Check data types

permits_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33888 entries, 0 to 33887
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   geometry                   33544 non-null  geometry
 1   permitnumber               33888 non-null  object  
 2   permitnumbercreateddate    33888 non-null  object  
 3   issuedate                  33888 non-null  object  
 4   permitelapseddays          33888 non-null  int64   
 5   projectvalue               33888 non-null  float64 
 6   typeofwork                 33888 non-null  object  
 7   address                    33823 non-null  object  
 8   projectdescription         33888 non-null  object  
 9   permitcategory             19337 non-null  object  
 10  applicant                  33888 non-null  object  
 11  applicantaddress           33802 non-null  object  
 12  propertyuse                33883 non-null  object  
 13  specificusecategory    

In [5]:
#Remove unnecessary columns

cols_to_keep = ['geometry','projectvalue','typeofwork','propertyuse','issueyear','geolocalarea']

permits_df = permits_df[cols_to_keep]

permits_df.head()

,geometry,projectvalue,typeofwork,propertyuse,issueyear,geolocalarea
0,POINT (-123.09119 49.22429),10000.0,Addition / Alteration,[Office Uses],2018,Sunset
1,POINT (-123.12880 49.28819),50000.0,Addition / Alteration,[Dwelling Uses],2021,West End
2,POINT (-123.13701 49.29288),60000.0,Addition / Alteration,[Dwelling Uses],2019,West End
3,POINT (-123.13715 49.28095),50000.0,Addition / Alteration,[Dwelling Uses],2021,West End
4,POINT (-123.13825 49.28318),35000.0,Addition / Alteration,[Dwelling Uses],2019,West End


In [6]:
#Rename columns

new_columns = {'projectvalue':'ProjectValue',
               'typeofwork':'WorkType',
               'propertyuse':'PropertyUse',
               'issueyear':'Year',
               'geolocalarea':'Area'
               }

permits_df.rename(columns=new_columns, inplace=True)

In [7]:
permits_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 33888 entries, 0 to 33887
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   geometry      33544 non-null  geometry
 1   ProjectValue  33888 non-null  float64 
 2   WorkType      33888 non-null  object  
 3   PropertyUse   33883 non-null  object  
 4   Year          33888 non-null  object  
 5   Area          33537 non-null  object  
dtypes: float64(1), geometry(1), object(4)
memory usage: 1.6+ MB


In [8]:
#Convert year to integer

permits_df['Year'] = permits_df['Year'].astype(str).astype(int)
permits_df.dtypes

geometry        geometry
ProjectValue     float64
WorkType          object
PropertyUse       object
Year               int32
Area              object
dtype: object

In [9]:
permits_df.head()

,geometry,ProjectValue,WorkType,PropertyUse,Year,Area
0,POINT (-123.09119 49.22429),10000.0,Addition / Alteration,[Office Uses],2018,Sunset
1,POINT (-123.12880 49.28819),50000.0,Addition / Alteration,[Dwelling Uses],2021,West End
2,POINT (-123.13701 49.29288),60000.0,Addition / Alteration,[Dwelling Uses],2019,West End
3,POINT (-123.13715 49.28095),50000.0,Addition / Alteration,[Dwelling Uses],2021,West End
4,POINT (-123.13825 49.28318),35000.0,Addition / Alteration,[Dwelling Uses],2019,West End


#### Data Cleaning & Preparation - Property Use

In [10]:
#Check value counts for property use

permits_df.PropertyUse.value_counts()

[Dwelling Uses]                                                                 23804
[Office Uses]                                                                    4206
[Retail Uses]                                                                    1684
[Service Uses]                                                                   1330
[Institutional Uses]                                                              667
                                                                                ...  
[Cultural/Recreational Uses, Dwelling Uses, Institutional Uses, Retail Uses]        1
[Dwelling Uses, Manufacturing Uses, Office Uses, Parking Uses]                      1
[Institutional Uses, Office Uses, Parking Uses, Retail Uses]                        1
[Office Uses, Parking Uses, Service Uses, Wholesale Uses]                           1
[Cultural/Recreational Uses, Parking Uses, Service Uses]                            1
Name: PropertyUse, Length: 130, dtype: int64

In [11]:
# Create a number of uses column

permits_df["NumberOfUses"] = permits_df.PropertyUse.str.len()

#Remove rows where the number of use is blank

permits_df = permits_df[permits_df['NumberOfUses'].notna()]

#Convert number of uses to integer

permits_df["NumberOfUses"] = permits_df["NumberOfUses"].astype(int)

In [12]:
permits_df.NumberOfUses.value_counts()

1    32891
2      733
3      186
4       57
5       16
Name: NumberOfUses, dtype: int64

In [13]:
#Create a new revised property use column that converts the original column from a list of values into a string of values

permits_df['PropertyUseRevised'] = permits_df.PropertyUse.apply(lambda x: ', '.join([str(i) for i in x]))

permits_df.head()

,geometry,ProjectValue,WorkType,PropertyUse,Year,Area,NumberOfUses,PropertyUseRevised
0,POINT (-123.09119 49.22429),10000.0,Addition / Alteration,[Office Uses],2018,Sunset,1,Office Uses
1,POINT (-123.12880 49.28819),50000.0,Addition / Alteration,[Dwelling Uses],2021,West End,1,Dwelling Uses
2,POINT (-123.13701 49.29288),60000.0,Addition / Alteration,[Dwelling Uses],2019,West End,1,Dwelling Uses
3,POINT (-123.13715 49.28095),50000.0,Addition / Alteration,[Dwelling Uses],2021,West End,1,Dwelling Uses
4,POINT (-123.13825 49.28318),35000.0,Addition / Alteration,[Dwelling Uses],2019,West End,1,Dwelling Uses


In [14]:
#Change property use revised column base on the number of uses column. If the number of uses is greater than 1, change the value of property use revised to mixed use, otherwise keep it the same.

permits_df['PropertyUseRevised'] = np.where(permits_df.NumberOfUses > 1, 'Mixed Uses', permits_df.PropertyUseRevised)

In [15]:
permits_df.PropertyUseRevised.value_counts()

Dwelling Uses                      23804
Office Uses                         4206
Retail Uses                         1684
Service Uses                        1330
Mixed Uses                           992
Institutional Uses                   667
Cultural/Recreational Uses           540
Manufacturing Uses                   250
Wholesale Uses                       126
Transportation and Storage Uses      112
Parking Uses                          92
Utility and Communication Uses        56
Live-Work Uses                        19
Agricultural Uses                      3
Mural                                  1
Deposition or Extraction Uses          1
Name: PropertyUseRevised, dtype: int64

In [16]:
#Remove number of uses and original property use columns now that they are redundant

permits_df = permits_df[['geometry','ProjectValue','WorkType','Year','Area','PropertyUseRevised']]

permits_df.head()

,geometry,ProjectValue,WorkType,Year,Area,PropertyUseRevised
0,POINT (-123.09119 49.22429),10000.0,Addition / Alteration,2018,Sunset,Office Uses
1,POINT (-123.12880 49.28819),50000.0,Addition / Alteration,2021,West End,Dwelling Uses
2,POINT (-123.13701 49.29288),60000.0,Addition / Alteration,2019,West End,Dwelling Uses
3,POINT (-123.13715 49.28095),50000.0,Addition / Alteration,2021,West End,Dwelling Uses
4,POINT (-123.13825 49.28318),35000.0,Addition / Alteration,2019,West End,Dwelling Uses


#### Data Cleaning & Preparation - Work Type

In [17]:
permits_df.WorkType.value_counts()

Addition / Alteration                   16446
New Building                             7724
Salvage and Abatement                    4890
Demolition / Deconstruction              4460
Temporary Building / Structure            318
Outdoor Uses (No Buildings Proposed)       45
Name: WorkType, dtype: int64

In [18]:
#Rename some of the values in the work type field

permits_df['WorkType'] = permits_df['WorkType'].replace({'Addition / Alteration':'Addition',
                                                         'Salvage and Abatement':'Salvage',
                                                         'Demolition / Deconstruction':'Demolition',
                                                         'Temporary Building / Structure':'Temporary Building',
                                                         'Outdoor Uses (No Buildings Proposed)':'Outdoor Use'
                                                         })

permits_df.WorkType.value_counts()

Addition              16446
New Building           7724
Salvage                4890
Demolition             4460
Temporary Building      318
Outdoor Use              45
Name: WorkType, dtype: int64

#### Data Cleaning & Preparation - Year

In [19]:
permits_df.Year.value_counts()

2018    6758
2017    6729
2019    5572
2022    5382
2021    5052
2020    4390
Name: Year, dtype: int64

In [20]:
#Filter the data down to permits that were submitted between the years 2017 and 2019

permits_df = permits_df[permits_df['Year'].between(2017, 2019)]
permits_df.Year.value_counts()

2018    6758
2017    6729
2019    5572
Name: Year, dtype: int64

### Check For Missing Values

In [21]:
#Check the number of null values in the dataframe

total_missing = permits_df.isnull().sum().sort_values(ascending=False)
percent = (permits_df.isnull().sum()/permits_df.isnull().count()).sort_values(ascending=False)*100
missing_data = pd.concat([total_missing, percent], axis=1, keys=['Total Missing', 'Percent'])
missing_data.head()

,Total Missing,Percent
Area,151,0.792277
geometry,147,0.771289
ProjectValue,0,0.000000
WorkType,0,0.000000
Year,0,0.000000


In [22]:
#Since there is such a small amount of missing data drop the rows where there are missing values

permits_df = permits_df.dropna()
permits_df.isnull().sum()

geometry              0
ProjectValue          0
WorkType              0
Year                  0
Area                  0
PropertyUseRevised    0
dtype: int64

# Working With Other Geospatial Datasets

In [23]:
# Drops all columns but the geometry column. This is used on the datasets where you want to return a boolean value.

def geo_column_only(url, col_name):
    r = requests.get(url)
    gdf = gpd.GeoDataFrame().from_features(r.json())    
    final_table_columns = ['geometry']
    geo_df_with_cols_removed = gdf.drop(columns=[col for col in gdf if col not in final_table_columns])
    geo_df_with_cols_removed[col_name] = 1
    return geo_df_with_cols_removed

#Function to perform a left spatial join on two geodataframes and drop the right_index column after the join has been performed

def join_reducer(left, right):
    sjoin = gpd.sjoin(left, right, how='left')
    sjoin.drop('index_right', axis=1, inplace=True)
    return sjoin

## Business Improvement Areas

In [24]:
bia_df = geo_column_only('https://opendata.vancouver.ca/api/v2/catalog/datasets/business-improvement-areas-bia/exports/geojson?limit=-1&offset=0&timezone=UTC', 'Business_Improvement_Area')
bia_df.head()

,geometry,Business_Improvement_Area
0,"POLYGON ((-123.07049 49.27667, -123.07035 49.2...",1
1,"MULTIPOLYGON (((-123.03095 49.28069, -123.0236...",1
2,"POLYGON ((-123.15707 49.23507, -123.15706 49.2...",1
3,"POLYGON ((-123.13964 49.21416, -123.14033 49.2...",1
4,"POLYGON ((-123.13844 49.26669, -123.13916 49.2...",1


In [25]:
#Spatial join permits data to BIA data

permits_df = join_reducer(permits_df,bia_df)
permits_df['Business_Improvement_Area']
permits_df.head()

,geometry,ProjectValue,WorkType,Year,Area,PropertyUseRevised,Business_Improvement_Area
0,POINT (-123.09119 49.22429),10000.0,Addition,2018,Sunset,Office Uses,NaN
2,POINT (-123.13701 49.29288),60000.0,Addition,2019,West End,Dwelling Uses,NaN
4,POINT (-123.13825 49.28318),35000.0,Addition,2019,West End,Dwelling Uses,NaN
5,POINT (-123.13865 49.28343),55000.0,Addition,2019,West End,Dwelling Uses,NaN
6,POINT (-123.12209 49.26302),50000.0,Addition,2017,Fairview,Office Uses,NaN


In [26]:
permits_df.Business_Improvement_Area.value_counts()

1.0    4146
Name: Business_Improvement_Area, dtype: int64